# HydroHomies Plots
In this notebook, the plots, figures and also some explanations or details about each of them are being presented.  

To clarify, plaase fallow this order:
- Title for each plot is mandatory
- Analysis must be written 
- Address of data should be mentioned

# Figures



### **Comparison of Dehydration and control of one person**

### Explanations: 

### Data: 

In [6]:
# the code
pass

### Analysis: 

---

### **Comparison of Dehydration of all participants**

### Explanations: 

#### Data: 

In [7]:
#code

### Analysis: 

---

### **Comparison of Rehydration of all participants**

### Explanations: 

### Data: 

In [8]:
#code

### Analysis: 

---